# Making Final DataFrame
In this notebook, I will present the pipeline that I made to make the dataframe used in the Deep/Machine Learning models.

## Introduction
This notebook is the source of the python version of the pipeline. This one is more complete, and it contains labelizing and making dummies. So if one does not really want to use this notebook for the whole process they can make the dataset using the python pipeline, then implement the last part of this piepline in the wy that they want.

In this Notebook, I will describe the stages that I took to make the two main dataframes of my study.

## Make Main DataFrame
In this section, I will describe the stages that I took to make the main dataset, that contains 255 patients for 12-month endpoint and 303 patients for 6-month endpoint.

In [ ]:
# Import packages
# General packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
def xer_label_maker(diagnosis): # For the endpoints

    if diagnosis in ['Heel erg', 'Nogal']:
        return 1

    elif diagnosis in ['Een beetje', 'Helemaal niet']:
        return 0
    
    else:
        return 2
    
def xer_label_maker1(diagnosis): # For the baseline

    if diagnosis in ['Een beetje', 'Heel erg', 'Nogal']:
        return 1

    elif diagnosis in ['Helemaal niet']:
        return 0
    
    else:
        return 0

def xer_label_maker2(diagnosis): # For the week1

    if diagnosis in ['Heel erg', 'Nogal']:
        return 2

    elif diagnosis in ['Een beetje']:
        return 1
    
    elif diagnosis in ['Helemaal niet']:
        return 0
    
    else:
        return 0

def sex_label_maker(diagnosis): 

    if diagnosis in ['Man']:
        return 1
    
    else:
        return 0

def xer_label_maker_sanne1(diagnosis): # For the baseline

    if diagnosis in [2.0, 3.0, 4.0]:
        return 1

    elif diagnosis in [1.0]:
        return 0
    
    else:
        return 2

def xer_label_maker_sanne2(diagnosis): # For the week1

    if diagnosis in [3.0, 4.0]:
        return 1

    elif diagnosis in [1.0, 2.0]:
        return 0
    
    else:
        return 2

In [ ]:
# Introduce the links
dlc_bsl_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Rf_bsl_dlc_total.xlsx'
dlc_wk3_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Rf_wk3_dlc_total.xlsx'
xer_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Xerostomia_dataset.xlsx'
dose_path  = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/DLC_RTDOSE1.xlsx'
extra_dlc_bsl_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Extra_Rf_bsl_dlc_total.xlsx'

In [ ]:
## Make week3 df ()
dlc_wk3_df = pd.read_excel(dlc_wk3_path).drop(columns=['Unnamed: 0'])

# Slice the necessary part of Xerostomia datset
xer_df = pd.read_excel(xer_path)
xer_df['UMCG'] = xer_df['UMCG'].fillna(0).astype(int)
xer_df = xer_df.loc[:,['UMCG', 'GESLACHT', 'LEEFTIJD', 'Modality_adjusted', 'HN35_Xerostomia_BSL',
                       'Loctum2', 'HN35_Xerostomia_W01', 'HN35_Xerostomia_M06', 'HN35_Xerostomia_M12',
                    'Submandibular_Dmean', 'Parotid_L_Dmean', 'Parotid_R_Dmean']].reset_index().drop(columns=['index'])

xer_df = xer_df[xer_df.UMCG.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index']) # Slice the part that we have in weekly CTs

## Read and reshape the dose df
dose_df = pd.read_excel(dose_path).drop(columns=['Unnamed: 0'])
dose_df = dose_df.loc[:,['ID', 'name', 'mean']]
# Use pivot to reshape the DataFrame
dose_df = dose_df.pivot(index='ID', columns='name', values='mean')
# Reset the index to make 'ID' a regular column
dose_df.reset_index(inplace=True)
# Rename the columns for clarity
dose_df.columns.name = None 
dose_df['OAR'] = dose_df.idxmin(axis=1)
dose_df['OAR'] = dose_df['OAR'].str.replace('DLC_', '')
dose_df['Contra_Dmean'] = dose_df.min(axis=1)
dose_df = dose_df.drop(columns=['DLC_Parotid_R', 'DLC_Parotid_L'])
dose_df = dose_df[dose_df.ID.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index']) # Slice the part that we have in weekly CTs

## Convert the dlc baseline datafram in a comparieble way.
dlc_bsl_df = pd.read_excel(dlc_bsl_path)
dlc_bsl_df['Unnamed: 0'] = dlc_bsl_df['Unnamed: 0'].fillna(method='ffill')
dlc_bsl_df.rename(columns= {'Unnamed: 0':'ID', 'Unnamed: 1': 'OAR'}, inplace=True)
dlc_bsl_df.ID = dlc_bsl_df.ID.astype(int)

## Concat extra bsl df with the original one.
extra_dlc_bsl_df = pd.read_excel(extra_dlc_bsl_path)
extra_dlc_bsl_df = extra_dlc_bsl_df.drop(columns=['Unnamed: 0'])
dlc_bsl_df = pd.concat([dlc_bsl_df, extra_dlc_bsl_df]).sort_values(by='ID').reset_index().drop(columns=['index'])
dlc_bsl_df = dlc_bsl_df[dlc_bsl_df.ID.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index']) # Slice the part that we have in weekly CTs

# Preparing DLC baseline and week3 surface area
dlc_wk3_df = dlc_wk3_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_wk3_dlc'}).reset_index().drop(columns=['index'])
dlc_bsl_df = dlc_bsl_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_bsl_dlc'}).reset_index().drop(columns=['index'])

dlc_bsl_final_df = pd.DataFrame()
dlc_wk3_final_df = pd.DataFrame()

for counter, raw in dose_df.iterrows():
    assistant_bsl_df = dlc_bsl_df[dlc_bsl_df.ID == raw.ID]
    assistant_bsl_df = assistant_bsl_df[assistant_bsl_df.OAR.str.contains(raw.OAR)]

    assistant_wk3_df = dlc_wk3_df[dlc_wk3_df.ID == raw.ID]
    assistant_wk3_df = assistant_wk3_df[assistant_wk3_df.OAR.str.contains(raw.OAR)]
    dlc_bsl_final_df = pd.concat([dlc_bsl_final_df,assistant_bsl_df])
    dlc_wk3_final_df = pd.concat([dlc_wk3_final_df,assistant_wk3_df])

dlc_bsl_final_df = dlc_bsl_final_df.reset_index().drop(columns=['index'])
dlc_bsl_final_df = dlc_bsl_final_df.rename(columns={'OAR': 'OAR_bsl_dlc'})
dlc_wk3_final_df = dlc_wk3_final_df.reset_index().drop(columns=['index'])
dlc_wk3_final_df = dlc_wk3_final_df.rename(columns={'OAR': 'OAR_wk3_dlc'})

# Assemble the final dataset
final_df = dlc_bsl_final_df.merge(dlc_wk3_final_df, on='ID', how='inner')
final_df = final_df.merge(dose_df, on='ID', how='inner')
final_df = final_df.merge(xer_df, left_on='ID', right_on='UMCG', how='inner')
final_df = final_df.drop(columns=['OAR_bsl_dlc', 'OAR_wk3_dlc', 'UMCG'])

# Make the delta_surface column
final_df['delta_surf_dlc'] = (final_df.surface_bsl_dlc - final_df.surface_wk3_dlc) / 100 

# Rename some of the labels
final_df = final_df.rename(columns={'GESLACHT': 'sex', 'LEEFTIJD': 'age',
                         'HN35_Xerostomia_BSL': 'xer_bsl', 'HN35_Xerostomia_W01': 'xer_wk1',
                         'HN35_Xerostomia_M06': 'xer_06', 'HN35_Xerostomia_M12': 'xer_12'})

# Make a copy of xer_bsl column for CITOR model (this one should be divided into three columns)
final_df['xer_bsl_citor'] = final_df['xer_bsl'].copy()

# convert the labels to 0 and 1
final_df.xer_bsl = [xer_label_maker1(diagnosis) for diagnosis in final_df.xer_bsl]
final_df.xer_bsl_citor = [xer_label_maker2(diagnosis) for diagnosis in final_df.xer_bsl_citor]
final_df.xer_wk1 = [xer_label_maker2(diagnosis) for diagnosis in final_df.xer_wk1]
final_df.xer_06 = [xer_label_maker(diagnosis) for diagnosis in final_df.xer_06]
final_df.xer_12 = [xer_label_maker(diagnosis) for diagnosis in final_df.xer_12]
final_df.sex = [sex_label_maker(diagnosis) for diagnosis in final_df.sex]


# Separate xer_wk1 column into 3 columns (Create dummy columns for xer_wk1)
dummy_columns = pd.get_dummies(final_df['xer_wk1'])
final_df = pd.concat([final_df, dummy_columns], axis=1) # Add it to the main Dataset
final_df = final_df.rename(columns={0: 'xer_wk1_not_at_all', 1: 'xer_wk1_little', 2: 'xer_wk1_moderate_to_severe'})

# Separate xer_bsl_citor column into 3 columns (Create dummy columns for xer_bsl)
dummy_columns = pd.get_dummies(final_df['xer_bsl_citor'])
final_df = pd.concat([final_df, dummy_columns], axis=1) # Add it to the main Dataset
final_df = final_df.rename(columns={0: 'xer_bsl_not_at_all', 1: 'xer_bsl_little', 2: 'xer_bsl_moderate_to_severe'})

# Convert parotid_l and Parotid_r doses into the proper form for CITOR
final_df['sqr_parotid_Dmean'] = np.sqrt(final_df.Parotid_L_Dmean) + np.sqrt(final_df.Parotid_R_Dmean)

# Make 12 and 6 month datasets
six_month_df = final_df[~(final_df.xer_06 == 2)]
six_month_df = six_month_df.reset_index().drop(columns=['index'])

twelve_month_df = final_df[~(final_df.xer_12 == 2)]
twelve_month_df = twelve_month_df.reset_index().drop(columns=['index'])

# Add a Split column
six_month_df_x = six_month_df.drop(columns=['xer_12'])
X_train, X_test, y_train, y_test = train_test_split(six_month_df_x, six_month_df.xer_06, test_size=0.15, random_state=42)

six_month_df['Split'] = ['train_val' if idd in list(X_train.ID) else 'test' for idd in six_month_df.ID]

twelve_month_df = twelve_month_df.drop(columns=['xer_06'])
X_train, X_test, y_train, y_test = train_test_split(twelve_month_df, twelve_month_df.xer_12, test_size=0.15, random_state=42)

twelve_month_df['Split'] = ['train_val' if idd in list(X_train.ID) else 'test' for idd in twelve_month_df.ID]

# Save the dataframes
twelve_month_df.to_excel('delta_rf_df_12.xlsx')
six_month_df.to_excel('delta_rf_6month.xlsx')

# Make the dataset of Delta-radiomics article (2019)
This dataset belong to the article of Delta-radiomics model (2019) that contains 68 patients.

In [ ]:
dlc_bsl_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Rf_bsl_dlc_sanne.xlsx'
dlc_wk3_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Rf_wk3_dlc_sanne1.xlsx'
mc_bsl_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Rf_bsl_mc_sanne.xlsx'
mc_wk3_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Rf_wk3_mc_sanne.xlsx'

xer_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/Xerostomia_dataset.xlsx'
dlc_dose_path  = '//zkh/appdata/RTDicom/Projectline_HNC_modelling/OPC_data/ART Hooman/Hooman_project_data/Sanne_dataset/DLC_baseline_RTSTRUCT/DLC_RTDOSE_SANNE.xlsx'
mc_dose_path  = '//zkh/appdata/RTDicom/Projectline_HNC_modelling/OPC_data/ART Hooman/Hooman_project_data/Sanne_dataset/DLC_baseline_RTSTRUCT/MC_RTDOSE_SANNE.xlsx'
label_path = 'C:/Users/BahrdoH/OneDrive - UMCG/Hooman/Models/Preprocessing/Delta_radiomics/Feature_extraction_factory/Radiomics_features/Sanne_dataset/endpoints_xer2.xls'

In [ ]:
# Read the dataframe.
mc_wk3_df = pd.read_excel(mc_wk3_path).drop(columns=['Unnamed: 0'])
mc_bsl_df = pd.read_excel(mc_bsl_path).drop(columns=['Unnamed: 0'])
dlc_wk3_df = pd.read_excel(dlc_wk3_path).drop(columns=['Unnamed: 0'])
dlc_bsl_df = pd.read_excel(dlc_bsl_path).drop(columns=['Unnamed: 0'])


# Preparing DLC baseline and week3 surface area
dlc_wk3_df = dlc_wk3_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_wk3_dlc'}).reset_index().drop(columns=['index'])
dlc_bsl_df = dlc_bsl_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_bsl_dlc'}).reset_index().drop(columns=['index'])
mc_wk3_df = mc_wk3_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_wk3_mc'}).reset_index().drop(columns=['index'])
mc_bsl_df = mc_bsl_df.loc[:, ['ID', 'OAR', 'original_shape_SurfaceArea']].rename(columns={'original_shape_SurfaceArea': 'surface_bsl_mc'}).reset_index().drop(columns=['index'])

# Slice the necessary part of Xerostomia datset
label_df = pd.read_excel(label_path)
label_df = label_df.loc[:,['umcgnum', 'XER_w', 'Pati_baseline', 'Pati_6m', 'Pati_12m',]].reset_index().drop(columns=['index'])
label_df = label_df[label_df.umcgnum.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index']) # Slice the part that we have in weekly CTs

## Read and reshape the dose df
dlc_dose_df = pd.read_excel(dlc_dose_path).drop(columns=['Unnamed: 0'])
dlc_dose_df = dlc_dose_df.loc[:,['ID', 'name', 'mean']]
# Use pivot to reshape the DataFrame
dlc_dose_df = dlc_dose_df.pivot(index='ID', columns='name', values='mean')
# Reset the index to make 'ID' a regular column
dlc_dose_df.reset_index(inplace=True)
# Rename the columns for clarity
dlc_dose_df.columns.name = None 
dlc_dose_df['OAR'] = dlc_dose_df.idxmin(axis=1)
dlc_dose_df['OAR'] = dlc_dose_df['OAR'].str.replace('DLC_', '')
dlc_dose_df['Contra_Dmean'] = dlc_dose_df.min(axis=1)
dlc_dose_df = dlc_dose_df.drop(columns=['DLC_Parotid_R', 'DLC_Parotid_L'])
dlc_dose_df = dlc_dose_df[dlc_dose_df.ID.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index']) # Slice the part that we have in weekly CTs

## Read and reshape the dose df (Manual contours) (RECHECK THIS IF YOU HAVE WRONG ANSWERS) ##################
mc_dose_df = pd.read_excel(mc_dose_path).drop(columns=['Unnamed: 0'])
mc_dose_df = mc_dose_df.loc[:,['ID', 'name', 'mean']]
mc_dose_df = mc_dose_df[mc_dose_df.name.str.contains('TA')]
# Use pivot to reshape the DataFrame
mc_dose_df = mc_dose_df.pivot(index='ID', columns='name', values='mean')
# Reset the index to make 'ID' a regular column
mc_dose_df.reset_index(inplace=True)
# Rename the columns for clarity
mc_dose_df.columns.name = None 
mc_dose_df['OAR'] = mc_dose_df.idxmin(axis=1)
mc_dose_df['OAR'] = mc_dose_df['OAR'].str.replace('DLC_', '')
mc_dose_df['Contra_Dmean'] = mc_dose_df.min(axis=1)

# mc_dose_df = mc_dose_df.drop(columns=['Parotid_L', 'Parotid_L_TA', 'Parotid_R', 'Parotid_R_TA', 'parotis_li', 'parotis_re'])
mc_dose_df = mc_dose_df.drop(columns=['Parotid_L_TA', 'Parotid_R_TA'])
mc_dose_df = mc_dose_df[mc_dose_df.ID.isin(dlc_wk3_df.ID)].reset_index().drop(columns=['index'])
mc_dose_df['OAR'] = mc_dose_df['OAR'].replace('parotis_re', 'Parotid_R')
mc_dose_df['OAR'] = mc_dose_df['OAR'].replace('parotis_li', 'Parotid_L')
# mc_dose_df['ID'] = mc_dose_df['ID'].replace(3925846, 3952846)


dlc_bsl_final_df = pd.DataFrame()
dlc_wk3_final_df = pd.DataFrame()
mc_bsl_final_df = pd.DataFrame()
mc_wk3_final_df = pd.DataFrame()

for counter, raw in dlc_dose_df.iterrows():
    assistant_bsl_df = dlc_bsl_df[dlc_bsl_df.ID == raw.ID]
    
    assistant_bsl_df = assistant_bsl_df[assistant_bsl_df.OAR.str.contains(raw.OAR)]
    assistant_wk3_df = dlc_wk3_df[dlc_wk3_df.ID == raw.ID]
    assistant_wk3_df = assistant_wk3_df[assistant_wk3_df.OAR.str.contains(raw.OAR)]
    dlc_bsl_final_df = pd.concat([dlc_bsl_final_df,assistant_bsl_df])
    dlc_wk3_final_df = pd.concat([dlc_wk3_final_df,assistant_wk3_df])

dlc_bsl_final_df = dlc_bsl_final_df.reset_index().drop(columns=['index'])
dlc_bsl_final_df = dlc_bsl_final_df.rename(columns={'OAR': 'OAR_bsl_dlc'})
dlc_wk3_final_df = dlc_wk3_final_df.reset_index().drop(columns=['index'])
dlc_wk3_final_df = dlc_wk3_final_df.rename(columns={'OAR': 'OAR_wk3_dlc'})

mc_bsl_final_rows = []
mc_wk3_final_rows = []

for counter, raw in mc_dose_df.iterrows():
    assistant_bsl_df = mc_bsl_df[(mc_bsl_df.ID == raw.ID) & (mc_bsl_df.OAR.str.contains(raw.OAR))]
    assistant_wk3_df = mc_wk3_df[(mc_wk3_df.ID == raw.ID) & (mc_wk3_df.OAR.str.contains(raw.OAR))]

    if assistant_wk3_df.shape[0] == 0:
        assistant_wk3_df = mc_wk3_df[(mc_wk3_df.ID == raw.ID) & (mc_wk3_df.OAR.str.contains(raw.OAR.replace('_TA', '')))]
    if assistant_bsl_df.shape[0] == 0:
        assistant_bsl_df = mc_bsl_df[(mc_bsl_df.ID == raw.ID) & (mc_bsl_df.OAR.str.contains(raw.OAR.replace('_TA', '')))]

    if raw.ID ==4107088:
        assistant_wk3_df = mc_wk3_df[(mc_wk3_df.ID == 4107088) & (mc_wk3_df.OAR.str.contains('parotis_li'))]

    if assistant_bsl_df.shape[0] > 1:
        for _, row in assistant_bsl_df.iterrows():
            if 'ta' in row.OAR.lower():
                mc_bsl_final_rows.append(row)
    else:
        for _, row in assistant_bsl_df.iterrows():
            mc_bsl_final_rows.append(row)

    if assistant_wk3_df.shape[0] > 1:
        for _, row in assistant_wk3_df.iterrows():

            if 'ta' in row.OAR.lower():
                mc_wk3_final_rows.append(row)
    else:
        for _, row in assistant_wk3_df.iterrows():
            
            mc_wk3_final_rows.append(row)

mc_bsl_final_df = pd.DataFrame(mc_bsl_final_rows)
mc_wk3_final_df = pd.DataFrame(mc_wk3_final_rows)

mc_bsl_final_df.reset_index(drop=True, inplace=True)
mc_bsl_final_df.rename(columns={'OAR': 'OAR_bsl_mc'}, inplace=True)
mc_wk3_final_df.reset_index(drop=True, inplace=True)
mc_wk3_final_df.rename(columns={'OAR': 'OAR_wk3_mc'}, inplace=True)



# Assemble the final dataset
final_df = dlc_bsl_final_df.merge(dlc_wk3_final_df, on='ID', how='inner')
# print(final_df.shape)
final_df = final_df.merge(label_df, left_on='ID', right_on='umcgnum', how='inner')
print(final_df.shape)
final_df = final_df.merge(mc_bsl_final_df, on='ID', how='inner')
# print(final_df.shape)
final_df = final_df.merge(mc_wk3_final_df, on='ID', how='inner')
# print(final_df.shape)
final_df = final_df.merge(dlc_dose_df, on='ID', how='inner')
# print(final_df.shape)
final_df = final_df.merge(mc_dose_df, on='ID', how='inner')
# print(final_df.shape)

# print(final_df.shape)
final_df = final_df.drop(columns=['OAR_bsl_dlc', 'OAR_wk3_dlc', 'umcgnum', 'OAR_wk3_mc', 'OAR_bsl_mc'])


# Make the delta_surface column
final_df['delta_surf_dlc'] = (final_df.surface_wk3_dlc - final_df.surface_bsl_dlc) / 100 #/ 100. 
final_df['delta_surf_mc'] = (final_df.surface_wk3_mc - final_df.surface_bsl_mc) / 100 #/1000. #normalizing_feature
final_df['delta_surf_mcdlc'] = (final_df.surface_wk3_dlc - final_df.surface_bsl_mc) / 100

# Rename some of the labels
final_df = final_df.rename(columns={'Pati_baseline': 'xer_bsl', 'XER_w': 'xer_wk1',
                                    'Pati_6m': 'xer_06', 'Pati_12m': 'xer_12',
                                    'OAR_x': 'OAR_dlc', 'OAR_y': 'OAR_mc',
                                    'Contra_Dmean_x': 'Contra_Dmean_dlc',
                                    'Contra_Dmean_y': 'Contra_Dmean_mc'})


# convert the labels to 0 and 1
final_df.xer_bsl = [xer_label_maker_sanne1(diagnosis) for diagnosis in final_df.xer_bsl]
final_df.xer_wk1 = [xer_label_maker_sanne1(diagnosis) for diagnosis in final_df.xer_wk1]
final_df.xer_06 = [xer_label_maker_sanne2(diagnosis) for diagnosis in final_df.xer_06]
final_df.xer_12 = [xer_label_maker_sanne2(diagnosis) for diagnosis in final_df.xer_12]

# Make 12 and 6 month datasets
six_month_df = final_df[~(final_df.xer_06 == 2)]
six_month_df = six_month_df.reset_index().drop(columns=['index'])

twelve_month_df = final_df[~(final_df.xer_12 == 2)]
twelve_month_df = twelve_month_df.reset_index().drop(columns=['index'])

# Sanne_6month_df1 is the best one
six_month_df.to_excel('Sanne_6month.xlsx')
twelve_month_df.to_excel('Sanne_12month.xlsx')